# Libraries import

In [1]:
import itertools
import os
import sys
from tqdm import tqdm

import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from boltons.iterutils import pairwise
from geopy.distance import vincenty
from osgeo import gdal
from ipykernel import kernelapp as app

%matplotlib notebook
import matplotlib.pyplot as plt

# Data folder overview

In [2]:
data_folder = "/Users/lena/Dropbox/Hackathon/rv"

In [19]:
#INPUT FILE
administrative_path = os.path.join(
    data_folder, 'data','admin', 'stluc_administrative.shp')

administrative = gpd.read_file(administrative_path)

edge_shapefile = os.path.join(
    data_folder, 'data','developments', 'NS Link Road option 1.shp')

edge_shapefile_2 = os.path.join(
    data_folder, 'data','developments', 'NS Link Road Option 2.shp')

#edge_shapefile = gpd.read_file(edge_shapefile)
#edge_shapefile = edge_shapefile.to_crs({'init': 'epsg:4326'})
#edge_shapefile.crs = {'init': 'epsg:4326'}

edge_id_column= 'id'

hazard_shapefile = os.path.join(
        data_folder, 'data', 'hazards', '1m_sea-level.shp')

In [25]:
intersections_roads_path = os.path.join(
    data_folder, 'results', 'intersections_road_NS_1_merged.csv')

roads_haz_percent_exp_path = os.path.join(
    data_folder, 'results', 'roads_hazards_NS_1_%_exp.csv')

In [41]:
def line_length(line, ellipsoid='WGS-84'):
    """Length of a line in meters, given in geographic coordinates.
    Adapted from https://gis.stackexchange.com/questions/4022/looking-for-a-pythonic-way-to-calculate-the-length-of-a-wkt-linestring#answer-115285
    Args:
        line: a shapely LineString object with WGS-84 coordinates.
        ellipsoid: string name of an ellipsoid that `geopy` understands (see http://geopy.readthedocs.io/en/latest/#module-geopy.distance).
    Returns:
        Length of line in kilometers.
    """
    if line.geometryType() == 'MultiLineString':
        return sum(line_length(segment) for segment in line)

    return sum(
        vincenty(tuple(reversed(a)), tuple(reversed(b)), ellipsoid=ellipsoid).kilometers
        for a, b in pairwise(line.coords)
    )

In [104]:
def networkedge_hazard_intersection(edge_shapefile, hazard_shapefile, output_shapefile,edge_id_column):
    """Intersect network edges and hazards and write results to shapefiles
    Parameters
    ----------
    edge_shapefile
        Shapefile of network LineStrings
    hazard_shapefile
        Shapefile of hazard Polygons
    output_shapefile
        String name of edge-hazard shapefile for storing results
    Outputs
    -------
    output_shapefile
        - edge_id - String name of intersecting edge ID
        - length - Float length of intersection of edge LineString and hazard Polygon
        - geometry - Shapely LineString geometry of intersection of edge LineString and hazard Polygon
    """
    print ('* Starting {} and {} intersections'.format(edge_shapefile,hazard_shapefile))
    #adjust which new road to assess
    line_gpd = gpd.read_file(edge_shapefile)
    #line_gpd.crs = {'init': 'epsg:4326'}
    line_gpd = line_gpd.to_crs({'init': 'epsg:4326'})
    poly_gpd = gpd.read_file(hazard_shapefile)
       # poly_gpd.crs = {'init': 'epsg:4326'}
    poly_gpd = poly_gpd.to_crs({'init': 'epsg:4326'})

    if len(line_gpd.index) > 0 and len(poly_gpd.index) > 0:
        line_gpd.columns = map(str.lower, line_gpd.columns)
        poly_gpd.columns = map(str.lower, poly_gpd.columns)

        line_bounding_box = line_gpd.total_bounds
        line_bounding_box_coord = list(itertools.product([line_bounding_box[0], line_bounding_box[2]], [
                                       line_bounding_box[1], line_bounding_box[3]]))
        line_bounding_box_geom = Polygon(line_bounding_box_coord)
        line_bounding_box_gpd = gpd.GeoDataFrame(pd.DataFrame(
            [[1], [line_bounding_box_geom]]).T, crs='epsg:4326')
        line_bounding_box_gpd.columns = ['ID', 'geometry']

        poly_bounding_box = poly_gpd.total_bounds
        poly_bounding_box_coord = list(itertools.product([poly_bounding_box[0], poly_bounding_box[2]], [
                                       poly_bounding_box[1], poly_bounding_box[3]]))
        poly_bounding_box_geom = Polygon(poly_bounding_box_coord)
        poly_bounding_box_gpd = gpd.GeoDataFrame(pd.DataFrame(
            [[1], [poly_bounding_box_geom]]).T, crs='epsg:4326')
        poly_bounding_box_gpd.columns = ['ID', 'geometry']

        poly_sindex = poly_bounding_box_gpd.sindex

        selected_polys = poly_bounding_box_gpd.iloc[list(
            poly_sindex.intersection(line_bounding_box_gpd['geometry'].iloc[0].bounds))]
        if len(selected_polys.index) > 0:
            data = []
            poly_sindex = poly_gpd.sindex
            for l_index, lines in line_gpd.iterrows():
                intersected_polys = poly_gpd.iloc[list(
                    poly_sindex.intersection(lines.geometry.bounds))]
                for p_index, poly in intersected_polys.iterrows():
                    if (lines['geometry'].intersects(poly['geometry']) is True) and (poly.geometry.is_valid is True):
                        if line_length(lines['geometry']) > 1e-3:
                            data.append({edge_id_column: lines[edge_id_column], 'length': 1000.0*line_length(lines['geometry'].intersection(
                                poly['geometry'])), 'geometry': lines['geometry'].intersection(poly['geometry'])})
                        else:
                            data.append({edge_id_column: lines[edge_id_column], 'length': 0, 'geometry': lines['geometry']})
            if data:
                intersections_data = gpd.GeoDataFrame(
                    data, columns=[edge_id_column, 'length', 'geometry'], crs='epsg:4326')
                #intersections_data = intersections_data.to_crs({'init':'epsg:4326'})
                intersections_data.to_file(output_shapefile)

                #del intersections_data
                
    #return intersections_data
    del line_gpd, poly_gpd

In [102]:
def load_hazard(data_folder, hazard_id):  
    hazard_path = os.path.join(
        data_folder, 'data', 'hazards', '{}.shp'.format(hazard_id))
    hazards = geopandas.read_file(hazard_path)
    
    #if hazards.crs != {'init': 'epsg:4326'}:
    #    hazards = hazards.to_crs({'init': 'epsg:4326'})
    return hazards

In [94]:
hazard_ids = ['1m_sea-level','4m_storm-surge','flashflooding','landslide_susceptibility']

In [96]:
all_intersections=[]
for hazard_id in hazard_ids:
    #OUTPUT FILE
    hazard_shapefile = os.path.join(
        data_folder, 'data', 'hazards', '{}.shp'.format(hazard_id))
    output_path = os.path.join(
        data_folder, 'results', 'intersect_road_NS_1_{}.shp'.format(hazard_id))
    networkedge_hazard_intersection(edge_shapefile, hazard_shapefile,  output_path, edge_id_column)
    #intersections_data = intersections_data.rename(columns={
    #    'length': hazard_id
    #})
    #all_intersections.append(intersections_data)
#all_intersections = pd.concat(all_intersections, axis=0, sort=False)
#all_intersections.to_csv(intersections_roads_path)
#all_intersections.head()

* Starting /Users/lena/Dropbox/Hackathon/rv/data/developments/NS Link Road option 1.shp and /Users/lena/Dropbox/Hackathon/rv/data/hazards/1m_sea-level.shp intersections
* Starting /Users/lena/Dropbox/Hackathon/rv/data/developments/NS Link Road option 1.shp and /Users/lena/Dropbox/Hackathon/rv/data/hazards/4m_storm-surge.shp intersections


/Users/lena/miniconda3/envs/nismod_int/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  from ipykernel import kernelapp as app


* Starting /Users/lena/Dropbox/Hackathon/rv/data/developments/NS Link Road option 1.shp and /Users/lena/Dropbox/Hackathon/rv/data/hazards/flashflooding.shp intersections
* Starting /Users/lena/Dropbox/Hackathon/rv/data/developments/NS Link Road option 1.shp and /Users/lena/Dropbox/Hackathon/rv/data/hazards/landslide_susceptibility.shp intersections


In [147]:
hazard_ids = ['4m_storm-surge','flashflooding','landslide_susceptibility']
all_intersections=[]
for hazard_id in hazard_ids:
    intersections_data = gpd.read_file(os.path.join(
        data_folder, 'results', 'intersect_road_NS_2_{}.shp'.format(hazard_id)))
    intersections_data = intersections_data.rename(columns={
        'length': hazard_id
    })
    all_intersections.append(intersections_data)
all_intersections = pd.concat(all_intersections, axis=0, sort=False)
all_intersections.to_csv(intersections_roads_path)
all_intersections.head()

,id,4m_storm-surge,geometry,flashflooding,landslide_susceptibility
0,1001,154.676128,(LINESTRING (-60.96923873466363 14.03459586946...,NaN,NaN
1,1001,22.436988,LINESTRING (-60.96941986887242 14.034695127377...,NaN,NaN
0,1001,NaN,LINESTRING (-60.925804134084 13.95862900740008...,5.146266,NaN
1,1001,NaN,LINESTRING (-60.92579851229083 13.958675196299...,5.146266,NaN
2,1001,NaN,LINESTRING (-60.92579289049766 13.958721385198...,5.146266,NaN


In [148]:
#more elegantly: pd.DataFrame({'roads': all_intersections.sum()})
roads_exp = pd.DataFrame(all_intersections.sum(), columns=['roads'])
roads_exp = pd.DataFrame.transpose(roads_exp)
roads_exp = roads_exp.drop(['id'], axis=1)
roads_exp

,4m_storm-surge,flashflooding,landslide_susceptibility
roads,177.113115,2919.521592,1154.692835


In [149]:
line_gpd = gpd.read_file(edge_shapefile)
line_gpd = line_gpd.to_crs({'init': 'epsg:4326'})
line_gpd_m = line_gpd.length *100000
line_gpd_m.sum()

32451.372980624743

In [150]:
roads_exp_per = roads_exp.copy()
for hazard_id in hazard_ids:
    roads_exp_per[hazard_id] = round((roads_exp[hazard_id]/ line_gpd_m.sum())*100)
    #roads_exp_per.append(roads_per)

In [151]:
roads_exp_per

,4m_storm-surge,flashflooding,landslide_susceptibility
roads,1.0,9.0,4.0
